# Networks

## Introduction 

This tutorial gives an overview of the microwave network analysis 
features of **skrf**.  For this tutorial, and the rest of the scikit-rf documentation, it is  assumed that **skrf** has been imported as `rf`. Whether or not you follow this convention in your own code is up to you.

In [ ]:
import skrf as rf
from pylab import *

If this produces an import error, please see [Installation ](Installation.ipynb).

## Creating Networks


**skrf** provides an object for a N-port microwave [Network](../api/network.rst). A [Network](../api/network.rst) can be created in a number of ways:
 - from a Touchstone file
 - from S-parameters
 - from Z-parameters
 - from other RF parameters (Y, ABCD, T, etc.) 
 
Some examples for each situation is given below.

### Creating Network from Touchstone file
[Touchstone file](https://en.wikipedia.org/wiki/Touchstone_file) (`.sNp` files, with `N` being the number of ports) is a _de facto_ standard to export N-port network parameter data and noise data of linear active devices, passive filters, passive devices, or interconnect networks. Creating a Network from a Touchstone file is simple:

In [ ]:
from skrf import Network, Frequency

ring_slot = Network('data/ring slot.s2p')

Note that some softwares, such as ANSYS HFSS, add additional information to the Touchstone standard, such as comments, simulation parameters, Port Impedance or Gamma (wavenumber). These data are also imported if detected. 

	
A short description of the network will be printed out if entered onto the command line
	

In [ ]:
ring_slot

### Creating Network from s-parameters
Networks can also be created by directly passing values for the `frequency`, `s`-parameters  (and optionally the port impedance `z0`). 

The scattering matrix of a N-port Network is expected to be a Numpy array of shape `(nb_f, N, N)`, where `nb_f` is the number of frequency points and `N` the number of ports of the network. 

In [ ]:
# dummy 2-port network from Frequency and s-parameters
freq = Frequency(1, 10, 101, 'ghz')
s = rand(101, 2, 2) + 1j*rand(101, 2, 2)  # random complex numbers 
# if not passed, will assume z0=50. name is optional but it's a good practice.
ntwk = Network(frequency=freq, s=s, name='random values 2-port') 
ntwk

In [ ]:
ntwk.plot_s_db()

Often, s-parameters are stored in separate arrays. In such case, one needs to forge the s-matrix:

In [ ]:
# let's assume we have separate arrays for the frequency and s-parameters
f = np.array([1, 2, 3, 4]) # in GHz
S11 = np.random.rand(4)
S12 = np.random.rand(4)
S21 = np.random.rand(4)
S22 = np.random.rand(4)

# Before creating the scikit-rf Network object, one must forge the Frequency and S-matrix:
freq2 = rf.Frequency.from_f(f, unit='GHz')

# forging S-matrix as shape (nb_f, 2, 2)
# there is probably smarter way, but less explicit for the purpose of this example:
s = np.zeros((len(f), 2, 2))
s[:,0,0] = S11
s[:,0,1] = S12
s[:,1,0] = S21
s[:,1,1] = S22

# constructing Network object
ntw = rf.Network(frequency=freq2, s=s)

print(ntw)

If necessary, the characteristic impedance can be passed as a scalar (same for all frequencies), as a list or an array:

In [ ]:
ntw2 = rf.Network(frequency=freq, s=s, z0=25, name='same z0 for all ports')
print(ntw2)
ntw3 = rf.Network(frequency=freq, s=s, z0=[20, 30], name='different z0 for each port')
print(ntw3)
ntw4 = rf.Network(frequency=freq, s=s, z0=rand(101,2), name='different z0 for each frequencies and ports')
print(ntw4)

### from z-parameters 

As networks are also defined from their Z-parameters, there is `from_z()` method of the Network:

In [ ]:
# 1-port network example
z = 10j
Z = np.full((len(freq), 1, 1), z)  # replicate z for all frequencies

ntw = rf.Network()
ntw = ntw.from_z(Z)
ntw.frequency = freq
print(ntw)

### from other network parameters (Z, Y, ABCD, T)
It is also possible to generate Networks from other kind of RF parameters, using the conversion functions: `z2s`, `y2s`, `a2s`, `t2s`, `h2s`. For example, the [ABCD parameters](https://en.wikipedia.org/wiki/Two-port_network#ABCD-parameters) of a serie-impedance is:
$$
\left[
\begin{array}{cc}
1 & Z \\
0 & 1
\end{array}
\right]
$$

In [ ]:
z = 20
abcd = array([[1, z],
              [0, 1]])

s = rf.a2s(tile(abcd, (len(freq),1,1)))
ntw = Network(frequency=freq, s=s)
print(ntw)

## Basic Properties

	
The basic attributes of a microwave [Network](../api/network.rst) are provided by the 
following properties :

* `Network.s` : Scattering Parameter matrix. 
* `Network.z0`  : Port Characteristic Impedance matrix.
* `Network.frequency`  : Frequency Object. 

The [Network](../api/network.rst) object has numerous other properties and methods. If you are using IPython, then these properties and methods can be 'tabbed' out on the command line. 


	In [1]: ring_slot.s<TAB>
	ring_slot.line.s              ring_slot.s_arcl         ring_slot.s_im
	ring_slot.line.s11            ring_slot.s_arcl_unwrap  ring_slot.s_mag
	...


All of the network parameters are represented internally as complex `numpy.ndarray`. The s-parameters are of shape (nfreq, nport, nport)

In [ ]:
shape(ring_slot.s)

## Slicing

You can  slice the `Network.s` attribute any way you want.

In [ ]:
ring_slot.s[:11,1,0]  # get first 10 values of S21

Slicing by frequency can also be done directly on Network objects like so 

In [ ]:
ring_slot[0:10] #  Network for the first 10 frequency points

or  with a human friendly string,

In [ ]:
ring_slot['80-90ghz']

Notice that slicing directly on a Network **returns a Network**.  So, a nice way to express slicing in both dimensions is 

In [ ]:
ring_slot.s11['80-90ghz'] 

## Plotting 

Amongst other things, the methods of the [Network](../api/network.rst) class provide convenient ways to plot components of the network parameters, 

* `Network.plot_s_db` : plot magnitude of s-parameters in log scale
* `Network.plot_s_deg` : plot phase of s-parameters in degrees
* `Network.plot_s_smith` : plot complex s-parameters on Smith Chart
* ...

If you would like to use skrf's plot styling,

In [ ]:
%matplotlib inline 
rf.stylely()

	
To plot all four s-parameters of the `ring_slot` on the Smith Chart.

In [ ]:
ring_slot.plot_s_smith()

Combining this with the slicing features, 

In [ ]:
from matplotlib import pyplot as plt

plt.title('Ring Slot $S_{21}$')

ring_slot.s11.plot_s_db(label='Full Band Response')
ring_slot.s11['82-90ghz'].plot_s_db(lw=3,label='Band of Interest')

For more detailed information about plotting see [Plotting](Plotting.ipynb).   

## Arithmetic Operations 
	
Element-wise mathematical operations on the scattering parameter matrices are accessible through overloaded operators. To illustrate their usage, load a couple  Networks stored in the `data` module. 

In [ ]:
from skrf.data import wr2p2_short as short 
from skrf.data import wr2p2_delayshort as delayshort 


short - delayshort
short + delayshort
short * delayshort
short / delayshort


All of these operations return [Network](../api/network.rst) types.  For example, to plot the complex difference  between  `short` and `delay_short`,

In [ ]:
difference = (short - delayshort)
difference.plot_s_mag(label='Mag of difference')

Another common application is calculating the phase difference using the division operator,

In [ ]:
(delayshort/short).plot_s_deg(label='Detrended Phase')

Linear operators can also be used with scalars or an `numpy.ndarray`  that ais the same length as the [Network](../api/network.rst). 

In [ ]:
hopen = (short*-1)
hopen.s[:3,...]

In [ ]:
rando =  hopen *rand(len(hopen))
rando.s[:3,...]

## Comparaisons of Network
Comparison operators also work with networks:

In [ ]:
short == delayshort

In [ ]:
short != delayshort

## Cascading and De-embedding

Cascading and de-embeding 2-port Networks can also be done though operators. The `cascade` function can be called through the power operator,  `**`. To calculate a new network which is the cascaded connection of the two individual Networks `line` and `short`, 

In [ ]:
short = rf.data.wr2p2_short
line = rf.data.wr2p2_line
delayshort = line ** short

De-embedding  can be accomplished by cascading the *inverse* of a network. The inverse of a network is accessed through the property `Network.inv`. To de-embed the `short` from `delay_short`,

In [ ]:
short_2 = line.inv ** delayshort

short_2 == short

The cascading operator also works for to 2N-port Networks. This is illustrated in [this example on balanced Networks](../examples/networktheory/Balanced%20Network%20De-embedding.ipynb). For example, assuming you want to cascade three 4-port Network `ntw1`, `ntw2` and `ntw3`, you can use:
`resulting_ntw = ntw1 ** ntw2 ** ntw3`. Note that the port scheme assumed by the `**` cascading operator with 4-port networks is:

```
   ntw1    ntw2    ntw3
  +----+  +----+  +----+
0-|0  2|--|0  2|--|0  2|-2
1-|1  3|--|1  3|--|1  3|-3
  +----+  +----+  +----+
```

## Connecting Multi-ports 

**skrf** supports the connection of arbitrary ports of N-port networks. It accomplishes this using an algorithm called sub-network growth[[1]](#References),  available through the function `connect()`. Terminating one port of an ideal 3-way splitter can be done like so,

In [ ]:
tee = rf.data.tee
tee

	

To connect port `1` of the tee, to port `0` of the delay short,

In [ ]:
terminated_tee = rf.connect(tee,1,delayshort,0)
terminated_tee

Note that this function takes into account port impedances. If two connected ports have different port impedances,  an appropriate impedance mismatch is inserted. 

For advanced connection between arbitrary N-port Networks, the `Circuit` object is more relevant since it allows defining explicitely the connections between ports and Networks. For more information, please refer to the [Circuit documentation](Circuit.ipynb). 

	
## Interpolation and Concatenation

A common need is to change the number of frequency points of a [Network](../api/network.rst). To use the operators and cascading functions the networks involved must have matching frequencies, for instance. If two networks have different frequency information, then an error will be raised, 

In [ ]:
from skrf.data import wr2p2_line1 as line1

line1

    line1+line
    
    ---------------------------------------------------------------------------
    IndexError                                Traceback (most recent call last)
    <ipython-input-49-82040f7eab08> in <module>()
    ----> 1 line1+line

    /home/alex/code/scikit-rf/skrf/network.py in __add__(self, other)
        500 
        501         if isinstance(other, Network):
    --> 502             self.__compatable_for_scalar_operation_test(other)
        503             result.s = self.s + other.s
        504         else:

    /home/alex/code/scikit-rf/skrf/network.py in __compatable_for_scalar_operation_test(self, other)
        701         '''
        702         if other.frequency  != self.frequency:
    --> 703             raise IndexError('Networks must have same frequency. See `Network.interpolate`')
        704 
        705         if other.s.shape != self.s.shape:

    IndexError: Networks must have same frequency. See `Network.interpolate`


	
This problem can be solved by interpolating one of Networks allong the frequency axis using `Network.resample`. 

In [ ]:
line1.resample(201)
line1

And now we can do things

In [ ]:
line1 + line

You can also interpolate from a `Frequency` object. For example, 

In [ ]:
line.interpolate_from_f(line1.frequency)

A related application is the need to combine Networks which cover different frequency ranges. Two  Netwoks can be concatenated (aka stitched) together using `stitch`, which  concatenates networks along their frequency axis. To combine a WR-2.2 Network with a WR-1.5 Network, 
 

In [ ]:
from skrf.data import wr2p2_line, wr1p5_line

big_line = rf.stitch(wr2p2_line, wr1p5_line)
big_line

## Reading and Writing 


For long term data storage, **skrf** has support for reading and partial support for writing  [touchstone file format](http://en.wikipedia.org/wiki/Touchstone_file). Reading is accomplished with the Network initializer as shown above,  and writing with the method  `Network.write_touchstone()`.

For **temporary** data storage, **skrf** object can be [pickled](http://docs.python.org/2/library/pickle.html) with  the functions `skrf.io.general.read` and `skrf.io.general.write`.  The reason to use temporary pickles over touchstones is that they store all attributes of a network, while touchstone files only store partial information. 

In [ ]:
rf.write('data/myline.ntwk',line) # write out Network using pickle

In [ ]:
ntwk = Network('data/myline.ntwk') # read Network using pickle

Frequently there is an entire directory of files that need to be analyzed. `rf.read_all`  creates Networks from all files in a directory quickly.  To load all **skrf** files in the `data/` directory which contain the string `'wr2p2'`.

In [ ]:
dict_o_ntwks = rf.read_all(rf.data.pwd, contains = 'wr2p2')
dict_o_ntwks

Other times you know the list of files that need to be analyzed. `rf.read_all`  also accepts a files parameter. This example file list contains only files within the same directory, but you can store files however your application would benefit from.

In [ ]:
import os
dict_o_ntwks_files = rf.read_all(files=[os.path.join(rf.data.pwd, test_file) for test_file in ['ntwk1.s2p', 'ntwk2.s2p']])
dict_o_ntwks_files

## Other Parameters	

This tutorial focuses on s-parameters, but other network representations are available as well. Impedance and Admittance Parameters can be accessed through the parameters `Network.z` and `Network.y`, respectively. Scalar components of complex parameters, such as  `Network.z_re`, `Network.z_im` and plotting methods are available as well.

Other parameters are only available for 2-port networks, such as wave cascading parameters (`Network.t`), and  ABCD-parameters (`Network.a`)

In [ ]:
ring_slot.z[:3,...]

In [ ]:
ring_slot.plot_z_im(m=1,n=0)

## References


[1] Compton, R.C.; , "Perspectives in microwave circuit analysis," Circuits and Systems, 1989., Proceedings of the 32nd Midwest Symposium on , vol., no., pp.716-718 vol.2, 14-16 Aug 1989. URL: http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=101955&isnumber=3167
